In [ ]:
import random

# Parámetros del algoritmo
NUM_HORMIGAS = 10  # Número de hormigas
NUM_ITERACIONES = 50  # Número de iteraciones
EVAPORACION = 0.5  # Tasa de evaporación de feromonas
ALFA = 1  # Peso de las feromonas en la decisión
BETA = 2  # Peso de la visibilidad (distancia inversa) en la decisión
Q = 100  # Cantidad de feromonas depositadas

# Grafo de ejemplo (matriz de distancias)
DISTANCIAS = [
    [0, 2, 9, 10],
    [2, 0, 6, 4],
    [9, 6, 0, 8],
    [10, 4, 8, 0]
]
NUM_NODOS = len(DISTANCIAS)

# Inicializar feromonas
feromonas = [[1 for _ in range(NUM_NODOS)] for _ in range(NUM_NODOS)]

# Función para calcular la visibilidad (inversa de la distancia)
def visibilidad(i, j):
    return 1 / DISTANCIAS[i][j] if DISTANCIAS[i][j] != 0 else 0

# Función para seleccionar el siguiente nodo
def seleccionar_siguiente_nodo(hormiga, nodos_visitados):
    nodo_actual = hormiga[-1]
    probabilidades = []
    for nodo in range(NUM_NODOS):
        if nodo not in nodos_visitados:
            feromona = feromonas[nodo_actual][nodo] ** ALFA
            vis = visibilidad(nodo_actual, nodo) ** BETA
            probabilidades.append((nodo, feromona * vis))
        else:
            probabilidades.append((nodo, 0))
    # Normalizar probabilidades
    total = sum(p for (n, p) in probabilidades)
    if total == 0:
        return random.choice([n for n in range(NUM_NODOS) if n not in nodos_visitados])
    probabilidades = [(n, p / total) for (n, p) in probabilidades]
    # Seleccionar nodo basado en probabilidades
    r = random.random()
    acumulado = 0
    for (nodo, prob) in probabilidades:
        acumulado += prob
        if r <= acumulado:
            return nodo
    return random.choice([n for n in range(NUM_NODOS) if n not in nodos_visitados])

# Función para calcular la longitud de una ruta
def calcular_longitud(ruta):
    return sum(DISTANCIAS[ruta[i]][ruta[i + 1]] for i in range(len(ruta) - 1))

# Algoritmo de colonia de hormigas
def colonia_hormigas():
    mejor_ruta = None
    mejor_longitud = float('inf')
    for iteracion in range(NUM_ITERACIONES):
        rutas = []
        for _ in range(NUM_HORMIGAS):
            # Iniciar en un nodo aleatorio
            nodo_inicial = random.randint(0, NUM_NODOS - 1)
            hormiga = [nodo_inicial]
            nodos_visitados = set([nodo_inicial])
            while len(hormiga) < NUM_NODOS:
                siguiente_nodo = seleccionar_siguiente_nodo(hormiga, nodos_visitados)
                hormiga.append(siguiente_nodo)
                nodos_visitados.add(siguiente_nodo)
            # Regresar al nodo inicial (ciclo cerrado)
            hormiga.append(hormiga[0])
            rutas.append(hormiga)
            # Actualizar la mejor ruta
            longitud = calcular_longitud(hormiga)
            if longitud < mejor_longitud:
                mejor_ruta = hormiga
                mejor_longitud = longitud
        # Evaporar feromonas
        for i in range(NUM_NODOS):
            for j in range(NUM_NODOS):
                feromonas[i][j] *= (1 - EVAPORACION)
        # Depositar feromonas en las rutas
        for ruta in rutas:
            longitud = calcular_longitud(ruta)
            for i in range(len(ruta) - 1):
                feromonas[ruta[i]][ruta[i + 1]] += Q / longitud
        print(f"Iteración {iteracion + 1}: Mejor longitud = {mejor_longitud}")
    return mejor_ruta, mejor_longitud

# Ejecutar el algoritmo
mejor_ruta, mejor_longitud = colonia_hormigas()
print(f"\nMejor ruta encontrada: {mejor_ruta}, Longitud: {mejor_longitud}")